In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Input
from keras.models import Model
from keras.layers import Dense, Flatten

In [15]:
import kagglehub

# Download latest version
path10 = kagglehub.dataset_download("liusha249/imagenet10")

print("Path to dataset files:", path10)

Path to dataset files: /root/.cache/kagglehub/datasets/ambityga/imagenet100/versions/8


In [14]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ambityga/imagenet100")

print("Path to dataset files:", path)

100%|██████████| 16.1G/16.1G [03:23<00:00, 85.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ambityga/imagenet100/versions/8


In [16]:
# Устанавливаем некоторые параметры для загрузки и обработки изображений
batch_size = 32
img_height = 224 # Вы можете выбрать другой размер
img_width = 224

# Создаем Dataset для обучения
train_ds = tf.keras.utils.image_dataset_from_directory(
    path10,
    labels='inferred',
    label_mode='categorical', # для многоклассовой классификации
    image_size=(img_height, img_width),
    batch_size=batch_size,
    validation_split=0.2, # Доля данных для валидации
    subset='training',
    seed=123
)

Found 13000 files belonging to 1 classes.
Using 10400 files for training.


In [18]:
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    return image, label

In [19]:
(X_train_10, y_train_10), (X_test_10, y_test_10) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [20]:
# Нормализация изображений
X_train_10 = X_train_10.astype("float32") / 255.0
X_test_10  = X_test_10.astype("float32") / 255.0

In [21]:
X_train_10 = X_train_10[:5000]
y_train_10 = y_train_10[:5000]

In [22]:
# Создание tf.data.Dataset с предварительной обработкой
train_ds_10 = (tf.data.Dataset.from_tensor_slices((X_train_10, y_train_10))
               .map(preprocess)
               .shuffle(buffer_size=10000)
               .batch(32))
test_ds_10  = (tf.data.Dataset.from_tensor_slices((X_test_10, y_test_10))
               .map(preprocess)
               .batch(32))

In [23]:
# Используем предварительно обученную модель MobileNetV2 без верхнего слоя
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [24]:
# Создание модели путем добавления новой классификационной головы
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)      # Вычисляем признаки
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)           # Добавляем Dropout для регуляризации
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)  # 10 классов для CIFAR-10
model_cifar10 = tf.keras.Model(inputs, outputs)

In [25]:
# Компиляция и обучение модели
model_cifar10.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

print("Модель для CIFAR-10:")
model_cifar10.summary()

model_cifar10.fit(train_ds_10, epochs=5, validation_data=test_ds_10)

Модель для CIFAR-10:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 661s 4s/step - accuracy: 0.4118 - loss: 1.7289 - val_accuracy: 0.7123 - val_loss: 0.8281
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 650s 4s/step - accuracy: 0.7253 - loss: 0.8061 - val_accuracy: 0.7468 - val_loss: 0.7303
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 686s 4s/step - accuracy: 0.7696 - loss: 0.6619 - val_accuracy: 0.7549 - val_loss: 0.7049
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 690s 4s/step - accuracy: 0.7926 - loss: 0.6100 - val_accuracy: 0.7585 - val_loss: 0.7004
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 682s 4s/step - accuracy: 0.8005 - loss: 0.5718 - val_accuracy: 0.7585 - val_loss: 0.6929


In [26]:
(X_train_100, y_train_100), (X_test_100, y_test_100) = tf.keras.datasets.cifar100.load_data(label_mode='fine')

X_train_100 = X_train_100.astype("float32") / 255.0
X_test_100  = X_test_100.astype("float32") / 255.0

X_train_100 = X_train_100[:5000]
y_train_100 = y_train_100[:5000]

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [27]:
# Создание tf.data.Dataset с предварительной обработкой
train_ds_100 = (tf.data.Dataset.from_tensor_slices((X_train_100, y_train_100))
                .map(preprocess)
                .shuffle(buffer_size=10000)
                .batch(32))
test_ds_100  = (tf.data.Dataset.from_tensor_slices((X_test_100, y_test_100))
                .map(preprocess)
                .batch(32))

In [28]:
# Создаем новую модель с той же базовой моделью (замороженной) и адаптированным количеством выходов
inputs_100 = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs_100, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs_100 = tf.keras.layers.Dense(100, activation='softmax')(x)  # 100 классов для CIFAR-100
model_cifar100 = tf.keras.Model(inputs_100, outputs_100)

In [ ]:
model_cifar100.compile(optimizer='adam',
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

print("Модель для CIFAR-100:")
model_cifar100.summary()

model_cifar100.fit(train_ds_100, epochs=5, validation_data=test_ds_100)

Модель для CIFAR-100:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │       128,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,386,084 (9.10 MB)

 Trainable params: 128,100 (500.39 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 662s 4s/step - accuracy: 0.1027 - loss: 4.3379 - val_accuracy: 0.3604 - val_loss: 2.5440
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 655s 4s/step - accuracy: 0.4506 - loss: 2.1256 - val_accuracy: 0.4302 - val_loss: 2.1850
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 631s 4s/step - accuracy: 0.5719 - loss: 1.6148 - val_accuracy: 0.4523 - val_loss: 2.0729
Epoch 4/5
119/157 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.6496 - loss: 1.3317